## Import Bible JSON

### Libraries

In [8]:
import urllib.request
import json
from typing import Union, List, Dict
from pathlib import Path

### Functions

In [2]:
def parse_json_from_url(url: str) ->  Union[list, dict]:
    response = urllib.request.urlopen(url)
    return json.loads(response.read())

### Execute

In [3]:
url = 'https://raw.githubusercontent.com/thiagobodruk/bible/master/json/en_bbe.json'
data = parse_json_from_url(url) # returns a list of dictionaries based on url

In [9]:
# Get the current working directory
cwd = Path.cwd()

# Get the parent of the working directory
parent_dir = cwd.parent

# Get the parent of the parent directory
grandparent_dir = parent_dir.parent

# Get path to /data/interim
data_interim_dir = grandparent_dir / Path('data') / Path('interim')

output_file_path = data_interim_dir / Path('en_bbe.json')


with open(output_file_path, 'w') as f:
    json.dump(data, f)

### Tests

#### Test Level 1: Type & Keys

In [5]:
print(type(data)) # List
print(type(data[0])) # Dict

<class 'list'>
<class 'dict'>


In [6]:
print(data[0].keys())

dict_keys(['abbrev', 'chapters', 'name'])


In [7]:

print(type(data[0]['name']))
print(type(data[0]['abbrev']))
print(type(data[0]['chapters']))

<class 'str'>
<class 'str'>
<class 'list'>


In [8]:
print(type(data[0]['chapters'][0]))
print(type(data[0]['chapters'][0][0]))

<class 'list'>
<class 'str'>


#### Test Level 2: Content

In [9]:
print(data[0]['chapters'][0][0])

At the first God made the heaven and the earth.


In [10]:
last_book = len(data)-1
last_chapter = len(data[last_book]['chapters'])-1
last_verse = len(data[last_book]['chapters'][last_chapter])-1
print(data[last_book]['chapters'][last_chapter][last_verse])

The grace of the Lord Jesus be with the saints. So be it.


#### Size Tests

In [11]:
print(len(data))

66


## Bible JSON to List of Dictionaries

### Libraries

In [12]:
import json
from typing import Union, List, Dict
from functools import reduce

### Functions

In [13]:
def get_verses_per_book(data: dict, book_number: int) -> dict:
    # Get the name of the book from the dictionary
    book_text = data[book_number]['name']
    reference_list = [] # keys of output dictionary
    verses_list = [] # values of output dictionary
    # Iterate over all chapters in the dictionary
    for chapter_number, chapter in enumerate(data[book_number]['chapters']):
        # Get the chapter number as a string
        chapter_number_txt = str(chapter_number+1)
        # Iterate over all verses in the chapter using enumerate() function
        for verse_number, verse in enumerate(chapter):
            # Get the verse number as a string
            verse_number_text = str(verse_number+1)
            # Append book name, chapter number and verse number into a string
            reference_list.append(f"{book_text} {chapter_number_txt}:{verse_number_text}")
            # Append verse to list
            verses_list.append(verse)   
    return  dict(zip(reference_list, verses_list))

In [14]:
def get_verses_of_all_books(data):
    bible_books = []
    for book_number, books in enumerate(data):
        book_verses = get_verses_per_book(data,book_number)
        bible_books.append(book_verses)
    return bible_books

In [15]:
def combine_verses_of_all_books(bible_books):
    return reduce(lambda a, b: dict(a, **b), bible_books)

### Execute

In [16]:
bible_books= get_verses_of_all_books(data)

In [17]:
bible_verses = combine_verses_of_all_books(bible_books)

### Tests

#### get_verses_of_all_books()

In [18]:
print(type(bible_books))
print(len(bible_books))
print(type(bible_books[0]))
print(bible_books[0])
print(bible_books[len(bible_books)-1])

<class 'list'>
66
<class 'dict'>
{'Genesis 1:1': 'At the first God made the heaven and the earth.', 'Genesis 1:2': 'And the earth was waste and without form; and it was dark on the face of the deep: and the Spirit of God was moving on the face of the waters.', 'Genesis 1:3': 'And God said, Let there be light: and there was light.', 'Genesis 1:4': 'And God, looking on the light, saw that it was good: and God made a division between the light and the dark,', 'Genesis 1:5': 'Naming the light, Day, and the dark, Night. And there was evening and there was morning, the first day.', 'Genesis 1:6': 'And God said, Let there be a solid arch stretching over the waters, parting the waters from the waters.', 'Genesis 1:7': 'And God made the arch for a division between the waters which were under the arch and those which were over it: and it was so.', 'Genesis 1:8': 'And God gave the arch the name of Heaven. And there was evening and there was morning, the second day.', 'Genesis 1:9': 'And God said,

#### combine_verses_of_all_books()

In [19]:
print(type(bible_verses))
print(len(bible_verses))

<class 'dict'>
31104


In [20]:
first_key, first_value = next(iter(bible_verses.items()))
print(f"{first_key} - {first_value}")

Genesis 1:1 - At the first God made the heaven and the earth.


In [21]:
last_key, last_value = list(bible_verses.items())[-1]
print(f"{last_key} - {last_value}")

Revelation 22:21 - The grace of the Lord Jesus be with the saints. So be it.


## Dictionaries to JSON

### Libraries

In [22]:
from pathlib import Path


### Filepath

In [23]:
# Get the current working directory
cwd = Path.cwd()

# Get the parent of the working directory
parent_dir = cwd.parent

# Get the parent of the parent directory
grandparent_dir = parent_dir.parent

# Get path to /data/interim
data_interim_dir = grandparent_dir / Path('data') / Path('interim')

### Functions

In [24]:
def dict_to_json(dictionary: dict) -> str:
    return json.dumps(dictionary, indent=4)

### Execute

#### Bible JSON

In [25]:
bible_verses_json = dict_to_json(bible_verses)

# Define the file path for the output file
output_file_path = data_interim_dir / Path('bible_verses.json')

# Write the JSON to the output file
with open(output_file_path, "w") as f:
    f.write(bible_verses_json)

#### Single Book JSON

In [28]:
file_name = data[0]['name'].lower().replace(" ", "")
file_path = data_interim_dir / Path('bible_books') / Path(f"{file_name}.json")

json_file = dict_to_json(bible_books[0])

with open(file_path, "w") as f:
    f.write(json_file)

#### Individual Bible Books JSON

In [29]:
for item in data:
    file_name = item['name'].lower().replace(" ", "")
    json_file = dict_to_json(bible_books[data.index(item)])
    file_path = data_interim_dir / Path('bible_books') / Path(f"{file_name}.json")
    with open(file_path, "w") as f:
        f.write(json_file)